<a href="https://cognitiveclass.ai"><img src = "https://raw.githubusercontent.com/googlecreativelab/quickdraw-dataset/master/preview.jpg" width = 600> </a>

## The Quick, Draw! Dataset
### <https://github.com/googlecreativelab/quickdraw-dataset>



In [ ]:
import os
import zipfile
local_zip = 'apple_banana_or_strawberry.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
base_dir = 'apple_banana_or_strawberry'

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'val')


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

img_path = "apple_banana_or_strawberry/train/apple/apple_0.tif"
img = mpimg.imread(img_path)
plt.imshow(img, cmap='gray')

plt.show()

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator()
test_datagen  = ImageDataGenerator()

# --------------------
# Flow training images in batches of 20 using train_datagen generator
# --------------------
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size=20,
                                                    class_mode='categorical',
                                                    color_mode='grayscale',
                                                    target_size=(28,28))
# --------------------
# Flow validation images in batches of 20 using test_datagen generator
# --------------------
validation_generator =  test_datagen.flow_from_directory(validation_dir,
                                                         batch_size=20,
                                                         class_mode  = 'categorical',
                                                         color_mode='grayscale',
                                                         target_size=(28,28))

In [ ]:
import tensorflow as tf

# model = tf.keras.models.Sequential([
#   tf.keras.layers.Flatten(input_shape=(28, 28)),
#   tf.keras.layers.Dense(128, activation=tf.nn.relu),
#   tf.keras.layers.Dense(3, activation=tf.nn.softmax)
# ])

model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(16, (5, 5), strides=(1, 1), activation='relu', input_shape=(28, 28, 1)),
  tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(100, activation='relu'),
  tf.keras.layers.Dense(3, activation='softmax')
])

model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss = 'categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

In [ ]:


callbacks = []

save_best_callback = tf.keras.callbacks.ModelCheckpoint(f'best_weights.hdf5', save_best_only=True, verbose=1)
callbacks.append(save_best_callback)

early_stop_callback = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True, verbose=1)
callbacks.append(early_stop_callback)

history = model.fit_generator(train_generator,
                              validation_data=validation_generator,
                              steps_per_epoch=50,
                              epochs=50,
                              validation_steps=15,
                              verbose=1,
                              callbacks=callbacks)

In [ ]:
test_datagen  = ImageDataGenerator()
test_dir = os.path.join(base_dir, 'test')
test_generator =  test_datagen.flow_from_directory(validation_dir,
                                                         batch_size=10,
                                                         class_mode  = 'categorical',
                                                         color_mode='grayscale',
                                                         target_size=(28,28))
loss, acc = model.evaluate(test_generator)
print(acc)